In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [2]:
def remove_par(df, feat):
    df[feat] = df[feat].str.replace('(','')
    df[feat] = df[feat].str.replace(')','')
    
def string2int(df, feat):
    df[feat] = df[feat].astype(int)
    
def polish_trade_feature(df, feat):
    remove_par(df, feat)
    string2int(df, feat)

In [3]:
# correct the name of the country to facilitate the merge betwen sambanis and geodata
dic_country_conv = dict({
    "Northern Alliance (Afghanistan)*":"Afghanistan",
    "South Vietnam":"Republic of Vietnam",
    'UAE':'United Arab Emirates',
    'UNITA (Angola)*':'Angola',
    'DR Congo':'Democratic Republic of the Congo',
    'United States':'United States of America',
    'Iran (Islamic Republic of)':'Iran',
    'Cote d’Ivoire':'Ivory Coast',
    "Lao People's Democratic Republic":'Laos',
    "Moldova (the Republic of)":'Moldova',
    'Viet Nam':'Republic of Vietnam',
    'Russian Federation':'Russia',
    'Korea (the Republic of)':'South Korea',
    'Eswatini':'Swaziland',
    'Syrian Arab Republic':'Syria',
    'Taiwan (Province of China)':'Taiwan',
    'Tanzania (the United Republic of)':'Tanzania',
    'United Kingdom of Great Britain and Northern Ireland':'United Kingdom',
    'Venezuela (Bolivarian Republic of)':'Venezuela',
    
    "Yemen People's Republic":'Yemen',
    'Yemen Arab Republic':'Yemen',
    '???':'Czechoslovakia',
    "???":'Yugoslavia',
    "???":'Zanzibar',
    "???":'Kosovo',
    "???":'Macedonia'
})
#left a space between the countries that required manual handling

#update the name of the countries in the geodata to allow a merge
#geodata["country_name"].replace(to_replace=dic_country_conv, inplace=True)
#geodata["country_border_name"].replace(to_replace=dic_country_conv, inplace=True)

In [4]:
trade = pd.read_csv("trade_reg.csv",sep='\t')

cols_sambanis = ["warstds", "ager", "agexp", "anoc", "army85", "autch98", "auto4",
        "autonomy", "avgnabo", "centpol3", "cowcode", "coldwar", "decade1", "decade2",
        "decade3", "decade4", "dem", "dem4", "demch98", "dlang", "drel",
        "durable", "ef", "ef2", "ehet", "elfo", "elfo2", "etdo4590",
        "expgdp", "exrec", "fedpol3", "fuelexp", "gdpgrowth", "geo1", "geo2",
        "geo34", "geo57", "geo69", "geo8", "illiteracy", "incumb", "infant",
        "inst", "inst3", "life", "lmtnest", "ln_gdpen", "lpopns", "major", "manuexp", "milper",
        "mirps0", "mirps1", "mirps2", "mirps3", "nat_war", "ncontig",
        "nmgdp", "nmdp4_alt", "numlang", "nwstate", "oil", "p4mchg",
        "parcomp", "parreg", "part", "partfree", "plural", "plurrel",
        "pol4", "pol4m", "pol4sq", "polch98", "polcomp", "popdense",
        "presi", "pri", "proxregc", "ptime", "reg", "regd4_alt", "relfrac", "seceduc",
        "second", "semipol3", "sip2", "sxpnew", "sxpsq", "tnatwar", "trade",
        "warhist", "xconst", "year", "country"]

sambanis = pd.read_csv("sambanis_extented.csv", usecols=cols_sambanis)

#keep only the year value for date (drop 1-january, dummy)
#sambanis["year"] = pd.DatetimeIndex(sambanis["year"]).year

#civil_war contains only the observations with a civil war
civil_war = sambanis[sambanis["warstds"]==1]

In [6]:
trade.loc[trade.recipient=="Viet Nam"]["no_ordered"].sum()
trade["recipient"].replace(to_replace=dic_country_conv, inplace=True)
trade["supplier"].replace(to_replace=dic_country_conv, inplace=True)
civil_war['year'] = pd.DatetimeIndex(civil_war['year']).year

In [7]:
import plotly.express as px
import plotly.graph_objects as go

In [14]:
types = {'aircraft':0, 'helicopter':0, 'gun':0, 'missile':0,
         'tank':0, 'engine':0, 'car':0, 'ac':0, 'apc':0}

def check_weapon_type(import_df):
    import_df["weapon_description"] = import_df[
                "weapon_description"].str.lower()
    
    # Get the wepon descriptions for given country's weapon import
    desc = import_df.weapon_description 
    tmp = types.copy()
    for t in tmp:
        idx = desc.loc[desc.str.count(t) == 1].index
        
        if t != 'missile':
          
            id_tmp = []
            anti_idx = desc.loc[desc.str.count('missile') == 1].index
            [id_tmp.append(x) for x in idx if x not in anti_idx]
            idx = id_tmp

        count = import_df.loc[idx].no_ordered.sum()

        tmp[t] = count
    total = import_df['no_ordered'].sum()
    tfound =sum(tmp.values())
    tmp['other'] = total-tfound

    return tmp


def imp_number(df, country, onset=None, buffer=5):
    
    country_imp = df.loc[trade.recipient == country]

    polish_trade_feature(country_imp, 'no_ordered')
    polish_trade_feature(country_imp, 'year_order')
    
    
    country_imp_pre = country_imp.loc[(country_imp.year_order < onset) & 
                                (country_imp.year_order > onset - buffer)]
    country_imp_post = country_imp.loc[(country_imp.year_order >= onset) & 
                                (country_imp.year_order < onset + buffer)]
    
 
    pre_onset_order = check_weapon_type(country_imp_pre).copy()
    pre_onset_order["when"]='pre onset'
    #total = country_imp_pre['no_ordered'].sum()

    #tfound =sum(pre_onset_order.values())

    #pre_onset_order["other"] = total - tfound
    
    
    
    
    post_onset_order = check_weapon_type(country_imp_post).copy()
    post_onset_order["when"]='post onset'
    
    return pd.DataFrame.from_dict([pre_onset_order]), pd.DataFrame.from_dict([post_onset_order])

In [15]:
t1, t2 = imp_number(trade, "Laos", onset = 1970)
ttotal = t1.append(t2)

In [16]:
fig = px.bar(ttotal,x="when", y=ttotal.columns, 
             labels=dict(x="Laos", y="imported weapons"))

fig.update_layout(
    width=500,
    height=600,
)
fig.update_yaxes(type="log")
fig.show()